# **University of Iowa Parking Pattern Analysis and Forecasting**

In order to run the reports in this notebook, install the following pre-requisites:

```bash
pip install dash
pip install jupyter-dash
pip install plotly

In [1]:
import pandas as pd
import numpy as np

# For interactive dashboard
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# For forecasting
from sklearn.linear_model import LinearRegression

## Table of Contents

1. [Load and prepare the datasets](#load-datasets-and-handle-missing-values)
2. [Monthly Usage Analysis](#monthly-usage-analysis)
3. [Usage Analysis by Lot, Access Group, and Hour of Day](#heatmap)
4. [Access Group Usage Analysis](#access-group-usage-analysis)
5. [Lot Number Usage Analysis](#lot-number-usage-analysis)
6. [Forecast for November and December 2024](#forecast-for-november-and-december-2024)

## Load datasets and handle missing values

In [2]:
# Load the datasets
access_group_df = pd.read_csv("CardAccessGroupAssignment.csv")
transaction_df = pd.read_csv("CardTransaction.csv")

# Convert EntranceTime and ExitTime to datetime
transaction_df['EntranceTime'] = pd.to_datetime(transaction_df['EntranceTime'], errors='coerce')
transaction_df['ExitTime'] = pd.to_datetime(transaction_df['ExitTime'], errors='coerce')

# Handle missing EntranceTime and ExitTime
transaction_df['EntranceTime'] = transaction_df.apply(
    lambda row: row['ExitTime'].replace(hour=0, minute=0, second=0) if pd.isnull(row['EntranceTime']) else row['EntranceTime'], 
    axis=1
)
transaction_df['ExitTime'] = transaction_df.apply(
    lambda row: row['EntranceTime'].replace(hour=23, minute=59, second=59) if pd.isnull(row['ExitTime']) else row['ExitTime'], 
    axis=1
)

## Prepare the data

In [3]:
# Calculate duration of parking (in hours)
transaction_df['Duration'] = (transaction_df['ExitTime'] - transaction_df['EntranceTime']).dt.total_seconds() / 3600

# Extract Month-Year and other date parts
transaction_df['MonthYear'] = transaction_df['EntranceTime'].dt.to_period('M').astype(str)
transaction_df['Year'] = transaction_df['EntranceTime'].dt.year
transaction_df['Month'] = transaction_df['EntranceTime'].dt.month_name()
transaction_df['Day'] = transaction_df['EntranceTime'].dt.day
transaction_df['Weekday'] = transaction_df['EntranceTime'].dt.day_name()
transaction_df['Date'] = transaction_df['EntranceTime'].dt.date
transaction_df['EntranceHour'] = transaction_df['EntranceTime'].dt.hour
transaction_df['ExitHour'] = transaction_df['ExitTime'].dt.hour
transaction_df['Hour'] = transaction_df['EntranceTime'].dt.hour

# Add a column for "All" in Access Groups
transaction_df['EffectiveGroupNumberWithAll'] = transaction_df['EffectiveGroupNumber'].fillna("Unknown").astype(str)

# Ensure LotNumber and GroupNumber are treated as string
transaction_df['LotNumber'] = transaction_df['LotNumber'].astype(str)  
transaction_df['EffectiveGroupNumber'] = transaction_df['EffectiveGroupNumber'].fillna("Unknown").astype(str)

## Monthly Usage Analysis

In [4]:
# Function to sort access groups numerically
def numeric_sort(value):
    try:
        return int(value)
    except ValueError:
        return float('inf')

# Sort access groups
transaction_df['EffectiveGroupNumberWithAll'] = pd.Categorical(
    transaction_df['EffectiveGroupNumberWithAll'],
    categories=sorted(transaction_df['EffectiveGroupNumberWithAll'].unique(), key=numeric_sort),
    ordered=True
)

# Unique years and months for dropdowns and radio buttons
unique_years = sorted(transaction_df['Year'].dropna().unique())
unique_months = [
    "All Months",  # Add "All Months" at the top
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# Initialize the Dash app
app = Dash(__name__)

# Layout for the dashboard
app.layout = html.Div([
    html.H1("Monthly Usage Analysis Dashboard"),
    
    # Container for all selection options in one line
    html.Div([
        # Year selection
        html.Label("Select Year:", style={'margin-right': '10px'}),
        dcc.RadioItems(
            id='year-radio',
            options=[{'label': 'All Years', 'value': 'All'}] + [{'label': year, 'value': year} for year in unique_years],
            value='All',  # Default to all years
            labelStyle={'display': 'inline-block', 'margin-right': '10px'},
            style={'margin-right': '20px'}
        ),

        # Month selection
        html.Label("Select Month:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': month, 'value': month} for month in unique_months],
            value='All Months',  # Default to "All Months"
            clearable=False,
            placeholder="Select a month",
            style={'width': '150px', 'display': 'inline-block', 'margin-right': '20px'}
        ),

        # Distribution selection
        html.Label("View Distribution By:", style={'margin-right': '10px'}),
        dcc.RadioItems(
            id='distribution-radio',
            options=[
                {'label': 'Access Groups', 'value': 'AccessGroup'},
                {'label': 'Lot Numbers', 'value': 'LotNumber'}
            ],
            value='AccessGroup',  # Default to Access Groups
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ], style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '20px'}),

    # Graph for combined analysis
    html.Div([
        html.H3("Monthly Usage Analysis"),
        dcc.Graph(id='combined-graph')
    ])
])

# Callback to update the graph based on the selected filters
@app.callback(
    Output('combined-graph', 'figure'),
    [
        Input('year-radio', 'value'),
        Input('month-dropdown', 'value'),
        Input('distribution-radio', 'value')
    ]
)
def update_graph(selected_year, selected_month, distribution_type):
    # Filter the data
    filtered_df = transaction_df.copy()
    if selected_year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == selected_year]
    if selected_month != 'All Months':
        filtered_df = filtered_df[filtered_df['Month'] == selected_month]

    # Define the column for grouping (AccessGroup or LotNumber)
    color_column = "EffectiveGroupNumberWithAll" if distribution_type == 'AccessGroup' else "LotNumber"

    # Define a function to create stacked bar charts for counts
    def create_stacked_chart(df, x_col, title):
        if df.empty:
            return px.bar(title="No data available")
        
        # Use px.histogram with `nbins` to count occurrences
        fig = px.histogram(
            df,
            x=x_col,
            color=color_column,
            title=title,
            barmode='stack',
            labels={color_column: 'Group' if distribution_type == 'AccessGroup' else 'Lot Number'},
            category_orders={x_col: sorted(df[x_col].unique())},
            histfunc="count"  # Aggregates by counting occurrences
        )
        fig.update_xaxes(title="Month-Year")
        fig.update_yaxes(title="Usage Count")
        return fig

    # Create the figure
    title = "Usage Distribution by " + ("Access Groups" if distribution_type == 'AccessGroup' else "Lot Numbers")
    return create_stacked_chart(filtered_df, 'MonthYear', title)

# Run the app
app.run_server(debug=True, port=8070)

## Usage Analysis by Lot, Access Group, and Hour of Day

In [5]:


# Sort lot numbers
sorted_lots = sorted(transaction_df['LotNumber'].unique(), key=lambda x: int(x) if x.isdigit() else float('inf'))

# Calculate the lot with the highest usage
highest_usage_lot = transaction_df['LotNumber'].value_counts().idxmax()

# Get unique years, months, and days for selection
unique_years = sorted(transaction_df['Year'].dropna().unique())
unique_months = [
    "All Months",
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
unique_days = ["All Days"] + sorted(transaction_df['Day'].dropna().unique())
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Initialize Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Usage Analysis by Lot, Access Group, and Hour of Day"),
    
    # First row: Year and Month selection
    html.Div([
        # Year selection
        html.Label("Select Year:", style={'font-size': '12px'}),
        dcc.RadioItems(
            id='year-radio',
            options=[{'label': 'All Years', 'value': 'All'}] + [{'label': year, 'value': year} for year in unique_years],
            value='All',  # Default to all years
            labelStyle={'display': 'inline-block', 'margin-right': '10px', 'font-size': '12px'},
            style={'margin-bottom': '10px'}
        ),

        # Month selection
        html.Label("Select Month:", style={'font-size': '12px'}),
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': month, 'value': month} for month in unique_months],
            value='All Months',  # Default to all months
            clearable=False,
            style={'width': '150px', 'margin-bottom': '10px', 'font-size': '12px'}
        )
    ], style={'display': 'flex', 'gap': '15px', 'flex-wrap': 'wrap'}),

    # Second row: Lot and Day of Month selection
    html.Div([
        # Lot selection
        html.Label("Select Lot Number:", style={'font-size': '12px'}),
        dcc.Dropdown(
            id='lot-dropdown',
            options=[{'label': lot, 'value': lot} for lot in sorted_lots],
            value=highest_usage_lot,  # Default to the lot with the highest usage
            clearable=False,
            style={'width': '150px', 'margin-bottom': '10px', 'font-size': '12px'}
        ),

        # Day selection
        html.Label("Select Day:", style={'font-size': '12px'}),
        dcc.Dropdown(
            id='day-dropdown',
            options=[{'label': day, 'value': day} for day in unique_days],
            value='All Days',  # Default to all days
            clearable=False,
            style={'width': '150px', 'margin-bottom': '10px', 'font-size': '12px'}
        )
    ], style={'display': 'flex', 'gap': '15px', 'flex-wrap': 'wrap'}),

    # Third row: Weekday checkboxes and Overnight filter
    html.Div([
        # Weekday checkboxes
        html.Label("Select Weekday(s):", style={'font-size': '12px'}),
        dcc.Checklist(
            id='weekday-checklist',
            options=[{'label': day, 'value': day} for day in weekdays],
            value=weekdays,  # Default to all days selected
            labelStyle={'display': 'inline-block', 'margin-right': '10px', 'font-size': '12px'},
            style={'margin-bottom': '10px'}
        ),

        # Overnight filter
        html.Label("Include Only Overnight:", style={'font-size': '12px'}),
        dcc.RadioItems(
            id='overnight-radio',
            options=[
                {'label': 'Yes', 'value': 'Overnight'},
                {'label': 'No', 'value': 'All'}
            ],
            value='All',  # Default to include all data
            labelStyle={'display': 'inline-block', 'margin-right': '10px', 'font-size': '12px'}
        )
    ], style={'display': 'flex', 'gap': '15px', 'flex-wrap': 'wrap'}),

    # Heatmap
    html.Div([
        dcc.Graph(id='usage-heatmap', style={'height': '500px', 'width': '700px'})
    ])
])

# Callback to update heatmap
@app.callback(
    Output('usage-heatmap', 'figure'),
    [
        Input('year-radio', 'value'),
        Input('month-dropdown', 'value'),
        Input('day-dropdown', 'value'),
        Input('weekday-checklist', 'value'),
        Input('lot-dropdown', 'value'),
        Input('overnight-radio', 'value')
    ]
)
def update_heatmap(selected_year, selected_month, selected_day, selected_weekdays, selected_lot, overnight_filter):
    # Filter data
    filtered_df = transaction_df.copy()
    if selected_year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == selected_year]
    if selected_month != 'All Months':
        filtered_df = filtered_df[filtered_df['Month'] == selected_month]
    if selected_day != 'All Days':
        filtered_df = filtered_df[filtered_df['Day'] == selected_day]
    if selected_weekdays:
        filtered_df = filtered_df[filtered_df['Weekday'].isin(selected_weekdays)]
    if overnight_filter == 'Overnight':
        filtered_df = filtered_df[filtered_df['Overnight'] == 1]  # Use the Overnight column directly
    filtered_df = filtered_df[filtered_df['LotNumber'] == selected_lot]

    # Aggregate data
    aggregated_df = filtered_df.groupby(['Hour', 'EffectiveGroupNumber']).size().reset_index(name='Usage')

    # Create the heatmap
    fig = px.density_heatmap(
        aggregated_df,
        x='Hour',
        y='EffectiveGroupNumber',
        z='Usage',
        color_continuous_scale='Viridis',
        title=f"Usage Analysis for Lot {selected_lot}",
        labels={'EffectiveGroupNumber': 'Access Group', 'Hour': 'Hour of Day', 'Usage': 'Usage Count'},
        nbinsx=24  # Ensures all 24 hours (0–23) are displayed without grouping
    )
    fig.update_xaxes(dtick=1)  # Show every hour on the x-axis
    fig.update_layout(height=500, width=700, xaxis_title="Hour of Day", yaxis_title="Access Group")
    return fig

# Run the app
app.run_server(debug=True, port=8075)


## Access Group Usage Analysis

In [6]:
# Function to sort access groups numerically where possible
def numeric_sort(value):
    try:
        return int(value)
    except ValueError:
        return float('inf')  # Place non-numeric values at the end

# Get unique values for dropdowns
unique_months = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
unique_access_groups = ["All"] + sorted(transaction_df['EffectiveGroupNumberWithAll'].unique(), key=numeric_sort)
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Access Group - Hour of Day Parking Patterns Dashboard"),
    
    # Container to hold year, month, day, and access group selection in a single line
    html.Div([
        # Year Selection
        html.Label("Year:", style={'margin-right': '10px'}),
        dcc.RadioItems(
            id='year-radio',
            options=[{'label': 'All Years', 'value': 'All'}] + [{'label': year, 'value': year} for year in unique_years],
            value='All',  # Default to 'All Years'
            labelStyle={'display': 'inline-block', 'margin-right': '10px'},
            style={'margin-right': '20px'}
        ),
        
        # Month Selection
        html.Label("Month:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': 'All Months', 'value': 'All'}] + [{'label': month, 'value': month} for month in unique_months],
            value='All',  # Default to 'All Months'
            clearable=False,
            placeholder="Select a month",
            style={'width': '120px', 'display': 'inline-block', 'margin-right': '20px'}
        ),
        
        # Day Selection
        html.Label("Day:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='day-dropdown',
            options=[{'label': 'All Days', 'value': 'All'}],  # Start with 'All Days' as default option
            placeholder="Select a day",
            clearable=True,
            style={'width': '100px', 'display': 'inline-block', 'margin-right': '20px'}
        ),

        # Access Group Selection
        html.Label("Access Group:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='group-dropdown',
            options=[{'label': group, 'value': group} for group in unique_access_groups],
            value='All',  # Default to 'All Groups'
            clearable=False,
            placeholder="Select a group",
            style={'width': '200px', 'display': 'inline-block'}
        )
    ], style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '20px'}),
    
    # Checkboxes for Weekdays
    html.Div([
        html.Label("Select Weekday(s):"),
        dcc.Checklist(
            id='weekday-checklist',
            options=[{'label': day, 'value': day} for day in weekdays],
            value=weekdays,  # Default to all days selected
            inline=True,
            labelStyle={'margin-right': '10px'}
        )
    ], style={'margin-bottom': '20px'}),

    # Checkbox for Overnight Filter
    html.Div([
        html.Label("Only Include Overnight Data:"),
        dcc.Checklist(
            id='overnight-checkbox',
            options=[{'label': 'Yes', 'value': 'Overnight'}],
            value=[],  # Default to not include overnight data
            inline=True,
            labelStyle={'margin-right': '10px'}
        )
    ], style={'margin-bottom': '20px'}),

    # Graph for Entrance Time
    html.Div([
        html.H3("Entrance Time Analysis"),
        dcc.Graph(id='entrance-time-graph')
    ]),

    # Graph for Exit Time
    html.Div([
        html.H3("Exit Time Analysis"),
        dcc.Graph(id='exit-time-graph')
    ])
])

# Callback to update the day dropdown based on selected month and year
@app.callback(
    Output('day-dropdown', 'options'),
    [Input('month-dropdown', 'value'), Input('year-radio', 'value')]
)
def update_day_options(month, year):
    # Filter data for the selected month and year to get valid days
    filtered_df = transaction_df
    if year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'All':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    
    unique_days = sorted(filtered_df['Day'].dropna().unique())  # Ensure numeric order
    
    # Add 'All Days' option at the beginning of the day list
    day_options = [{'label': 'All Days', 'value': 'All'}] + [{'label': day, 'value': day} for day in unique_days]
    return day_options

# Callback to update both Entrance Time and Exit Time graphs
@app.callback(
    [Output('entrance-time-graph', 'figure'), Output('exit-time-graph', 'figure')],
    [
        Input('month-dropdown', 'value'),
        Input('year-radio', 'value'),
        Input('day-dropdown', 'value'),
        Input('group-dropdown', 'value'),
        Input('weekday-checklist', 'value'),
        Input('overnight-checkbox', 'value')
    ]
)
def update_peak_usage(month, year, day, group, selected_weekdays, overnight_filter):
    # Filter data for the selected year, month, day, group, weekdays, and overnight filter
    filtered_df = transaction_df
    if year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'All':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    if day != 'All' and day is not None:
        filtered_df = filtered_df[filtered_df['Day'] == day]
    if group != 'All':
        filtered_df = filtered_df[filtered_df['EffectiveGroupNumberWithAll'] == group]
    if selected_weekdays:
        filtered_df = filtered_df[filtered_df['Weekday'].isin(selected_weekdays)]
    if 'Overnight' in overnight_filter:
        filtered_df = filtered_df[filtered_df['Overnight'] == 1]

    # Define a function to create a histogram
    def create_histogram(df, time_column, title):
        if df.empty:
            # Create an empty figure with a no-data message
            fig = go.Figure()
            fig.add_annotation(
                #text="No data available",
                text=f"No data available for {month} {year}",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False,
                font=dict(size=20, color="red")
            )
            fig.update_layout(title=title)
            return fig

        # Create a stacked bar chart
        fig = px.histogram(
            df,
            x=time_column,
            color="EffectiveGroupNumberWithAll",
            title=title,
            labels={'EffectiveGroupNumberWithAll': 'Access Group Number'},
            barmode='stack'
        )
        fig.update_xaxes(title="Hour of the Day")
        fig.update_yaxes(title="Number of Entries")
        return fig

    # Create figures for Entrance and Exit Times
    entrance_fig = create_histogram(filtered_df, 'EntranceHour', "Peak Usage by Entrance Time")
    exit_fig = create_histogram(filtered_df, 'ExitHour', "Peak Usage by Exit Time")
    return entrance_fig, exit_fig

# Run the app directly in Jupyter Notebook
app.run_server(mode='inline', port=8071)

## Lot Number - Hour of Day Usage Analysis

In [7]:
# Function to sort lot numbers numerically where possible
def numeric_sort(value):
    try:
        return int(value)
    except ValueError:
        return float('inf')  # Place non-numeric values at the end

# Get unique values for dropdowns
unique_months = [
    "All Months",  # Add "All Months" at the top
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
unique_lots = sorted(transaction_df['LotNumber'].dropna().unique(), key=numeric_sort)

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Lot Number - Hour of Day Usage Analysis Dashboard"),
    
    # Filter options
    html.Div([
        # Year Selection
        html.Label("Year:", style={'margin-right': '10px'}),
        dcc.RadioItems(
            id='year-radio',
            options=[{'label': 'All Years', 'value': 'All'}] + [{'label': year, 'value': year} for year in unique_years],
            value='All',  # Default to 'All Years'
            labelStyle={'display': 'inline-block', 'margin-right': '10px'},
            style={'margin-right': '20px'}
        ),
        
        # Month Selection
        html.Label("Month:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': 'All Months', 'value': 'All'}] + [{'label': month, 'value': month} for month in unique_months],
            value='All',  # Default to 'All Months'
            clearable=False,
            placeholder="Select a month",
            style={'width': '120px', 'display': 'inline-block', 'margin-right': '20px'}
        ),
        
        # Day Selection
        html.Label("Day:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='day-dropdown',
            options=[{'label': 'All Days', 'value': 'All'}],  # Start with 'All Days' as default option
            placeholder="Select a day",
            clearable=True,
            style={'width': '100px', 'display': 'inline-block', 'margin-right': '20px'}
        )
    ], style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '20px'}),
    
    # Weekday Selection
    html.Div([
        html.Label("Select Weekday(s):"),
        dcc.Checklist(
            id='weekday-checklist',
            options=[{'label': day, 'value': day} for day in weekdays],
            value=weekdays,  # Default to all days selected
            inline=True,
            labelStyle={'margin-right': '10px'}
        )
    ], style={'margin-bottom': '20px'}),

    # Graph for Lot Distribution by Entrance Time
    html.Div([
        html.H3("Lot Distribution by Entrance Time"),
        dcc.Graph(id='entrance-lot-graph')
    ]),

    # Graph for Lot Distribution by Exit Time
    html.Div([
        html.H3("Lot Distribution by Exit Time"),
        dcc.Graph(id='exit-lot-graph')
    ])
])

# Callback to update the day dropdown based on selected month and year
@app.callback(
    Output('day-dropdown', 'options'),
    [Input('month-dropdown', 'value'), Input('year-radio', 'value')]
)
def update_day_options(month, year):
    # Filter data for the selected month and year to get valid days
    filtered_df = transaction_df
    if year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'All':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    
    unique_days = sorted(filtered_df['Day'].dropna().unique())  # Ensure numeric order
    
    # Add 'All Days' option at the beginning of the day list
    day_options = [{'label': 'All Days', 'value': 'All'}] + [{'label': day, 'value': day} for day in unique_days]
    return day_options

# Callback to update Entrance and Exit Lot Distribution graphs
@app.callback(
    [Output('entrance-lot-graph', 'figure'), Output('exit-lot-graph', 'figure')],
    [
        Input('month-dropdown', 'value'),
        Input('year-radio', 'value'),
        Input('day-dropdown', 'value'),
        Input('weekday-checklist', 'value')
    ]
)
def update_lot_usage(month, year, day, selected_weekdays):
    # Filter data for the selected year, month, day, and weekdays
    filtered_df = transaction_df
    if year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'All':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    if day != 'All' and day is not None:
        filtered_df = filtered_df[filtered_df['Day'] == day]
    if selected_weekdays:
        filtered_df = filtered_df[filtered_df['Weekday'].isin(selected_weekdays)]

    # Create histograms for Lot Distribution
    def create_lot_histogram(df, time_column, title):
        if df.empty:
            # Create an empty figure with a no-data message
            fig = go.Figure()
            fig.add_annotation(
                text="No data available",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False,
                font=dict(size=20, color="red")
            )
            fig.update_layout(title=title)
            return fig

        # Create a stacked bar chart
        fig = px.histogram(
            df,
            x=time_column,
            color="LotNumber",
            title=title,
            labels={'LotNumber': 'Lot Number'},
            barmode='stack'
        )
        fig.update_xaxes(title="Hour of the Day")
        fig.update_yaxes(title="Usage Count")
        return fig

    entrance_lot_fig = create_lot_histogram(filtered_df, 'EntranceHour', "Lot Distribution by Entrance Time")
    exit_lot_fig = create_lot_histogram(filtered_df, 'ExitHour', "Lot Distribution by Exit Time")
    return entrance_lot_fig, exit_lot_fig

# Run the app directly in Jupyter Notebook
app.run_server(mode='inline', port=8072)

## Lot Number - Group Access Usage Analysis Dashboard

In [8]:
# Add a column for access groups
transaction_df['EffectiveGroupNumberWithAll'] = transaction_df['EffectiveGroupNumber'].fillna("Unknown").astype(str)

# Function to sort lot numbers numerically where possible
def numeric_sort(value):
    try:
        return int(value)
    except ValueError:
        return float('inf')  # Place non-numeric values at the end

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Lot Number - Group Access Usage Analysis Dashboard"),
    
    # Filter options
    html.Div([
        # Year Selection
        html.Label("Year:", style={'margin-right': '10px'}),
        dcc.RadioItems(
            id='year-radio',
            options=[{'label': 'All Years', 'value': 'All'}] + [{'label': year, 'value': year} for year in unique_years],
            value='All',  # Default to 'All Years'
            labelStyle={'display': 'inline-block', 'margin-right': '10px'},
            style={'margin-right': '20px'}
        ),
        
        # Month Selection
        html.Label("Month:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': 'All Months', 'value': 'All'}] + [{'label': month, 'value': month} for month in unique_months],
            value='All',  # Default to 'All Months'
            clearable=False,
            placeholder="Select a month",
            style={'width': '120px', 'display': 'inline-block', 'margin-right': '20px'}
        ),
        
        # Day Selection
        html.Label("Day:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='day-dropdown',
            options=[{'label': 'All Days', 'value': 'All'}],  # Start with 'All Days' as default option
            placeholder="Select a day",
            clearable=True,
            style={'width': '100px', 'display': 'inline-block', 'margin-right': '20px'}
        ),
        
        # Top Records Selection
        html.Label("Show Top Records:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='top-records-dropdown',
            options=[
                {'label': 'Top 5', 'value': 5},
                {'label': 'Top 10', 'value': 10},
                {'label': 'Top 15', 'value': 15},
                {'label': 'All', 'value': 'All'}
            ],
            value='All',  # Default to showing all records
            clearable=False,
            placeholder="Select number of top records",
            style={'width': '120px', 'display': 'inline-block', 'margin-right': '20px'}
        )
    ], style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '20px'}),
    
    # Weekday Selection
    html.Div([
        html.Label("Select Weekday(s):"),
        dcc.Checklist(
            id='weekday-checklist',
            options=[{'label': day, 'value': day} for day in weekdays],
            value=weekdays,  # Default to all days selected
            inline=True,
            labelStyle={'margin-right': '10px'}
        )
    ], style={'margin-bottom': '20px'}),

    # Graph for Lot Distribution by Entrance Time
    html.Div([
        html.H3("Lot Distribution by Entrance Time"),
        dcc.Graph(id='entrance-lot-graph')
    ]),

    # Graph for Lot Distribution by Exit Time
    html.Div([
        html.H3("Lot Distribution by Exit Time"),
        dcc.Graph(id='exit-lot-graph')
    ])
])

# Callback to update the day dropdown based on selected month and year
@app.callback(
    Output('day-dropdown', 'options'),
    [Input('month-dropdown', 'value'), Input('year-radio', 'value')]
)
def update_day_options(month, year):
    # Filter data for the selected month and year to get valid days
    filtered_df = transaction_df
    if year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'All':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    
    unique_days = sorted(filtered_df['Day'].dropna().unique())  # Ensure numeric order
    
    # Add 'All Days' option at the beginning of the day list
    day_options = [{'label': 'All Days', 'value': 'All'}] + [{'label': day, 'value': day} for day in unique_days]
    return day_options

# Callback to update Entrance and Exit Lot Distribution graphs
@app.callback(
    [Output('entrance-lot-graph', 'figure'), Output('exit-lot-graph', 'figure')],
    [
        Input('month-dropdown', 'value'),
        Input('year-radio', 'value'),
        Input('day-dropdown', 'value'),
        Input('weekday-checklist', 'value'),
        Input('top-records-dropdown', 'value')
    ]
)
def update_lot_usage(month, year, day, selected_weekdays, top_records):
    # Filter data for the selected year, month, day, and weekdays
    filtered_df = transaction_df
    if year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'All':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    if day != 'All' and day is not None:
        filtered_df = filtered_df[filtered_df['Day'] == day]
    if selected_weekdays:
        filtered_df = filtered_df[filtered_df['Weekday'].isin(selected_weekdays)]
    
    # Aggregate data for Lot Distribution
    lot_usage = filtered_df.groupby(['LotNumber', 'EffectiveGroupNumberWithAll']).size().reset_index(name='UsageCount')
    lot_usage = lot_usage.sort_values(by='UsageCount', ascending=False)

    # Filter top records
    if top_records != 'All':
        top_lots = lot_usage.groupby('LotNumber')['UsageCount'].sum().nlargest(top_records).index
        lot_usage = lot_usage[lot_usage['LotNumber'].isin(top_lots)]

    # Create stacked bar charts for Lot Distribution
    entrance_fig = px.bar(
        lot_usage,
        x='LotNumber',
        y='UsageCount',
        color='EffectiveGroupNumberWithAll',
        title="Top Lots by Entrance Count (Access Group Distribution)",
        labels={'LotNumber': 'Lot Number', 'UsageCount': 'Usage Count', 'EffectiveGroupNumberWithAll': 'Access Group'},
        barmode='stack'
    )

    exit_fig = px.bar(
        lot_usage,
        x='LotNumber',
        y='UsageCount',
        color='EffectiveGroupNumberWithAll',
        title="Top Lots by Exit Count (Access Group Distribution)",
        labels={'LotNumber': 'Lot Number', 'UsageCount': 'Usage Count', 'EffectiveGroupNumberWithAll': 'Access Group'},
        barmode='stack'
    )
    return entrance_fig, exit_fig

# Run the app directly in Jupyter Notebook
app.run_server(mode='inline', port=8073)

## Forecast for November and December 2024

In [9]:
# Extract daily peak usage for each lot
daily_peak_usage = transaction_df.groupby(['Date', 'LotNumber'])['CardNumber'].count().reset_index()
daily_peak_usage.rename(columns={'CardNumber': 'PeakUsage'}, inplace=True)

# Prepare forecast results
forecast_results = []

for lot in daily_peak_usage['LotNumber'].unique():
    # Filter data for the current lot
    lot_data = daily_peak_usage[daily_peak_usage['LotNumber'] == lot].copy()
    
    # Convert dates to numerical values
    lot_data['DaysSinceStart'] = (pd.to_datetime(lot_data['Date']) - pd.to_datetime(lot_data['Date'].min())).dt.days
    
    # Train the linear regression model
    X = lot_data[['DaysSinceStart']].values
    y = lot_data['PeakUsage'].values
    model = LinearRegression()
    model.fit(X, y)
    
    # Generate future dates
    future_dates = pd.date_range(start="2024-11-01", end="2024-12-31")
    future_days = (future_dates - pd.to_datetime(lot_data['Date'].min())).days
    future_X = future_days.values.reshape(-1, 1)
    
    # Predict peak usage for future dates
    future_predictions = model.predict(future_X)
    future_df = pd.DataFrame({
        'Date': future_dates,
        'PredictedPeakUsage': future_predictions,
        'LotNumber': lot
    })
    
    forecast_results.append(future_df)

# Combine all forecast data
forecast_df = pd.concat(forecast_results)

# Initialize Dash app
app = Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Parking Lot Usage Forecast"),
    
    # Dropdown for lot selection
    html.Div([
        html.Label("Select Lot:"),
        dcc.Dropdown(
            id='lot-dropdown',
            options=[{'label': str(lot), 'value': lot} for lot in daily_peak_usage['LotNumber'].unique()],
            value=daily_peak_usage['LotNumber'].unique()[0],  # Default to the first lot
            clearable=False,
            style={'width': '50%'}
        )
    ], style={'margin-bottom': '20px'}),
    
    # Graph for historical and forecasted data
    dcc.Graph(id='forecast-graph')
])

# Callback to update the graph based on the selected lot
@app.callback(
    Output('forecast-graph', 'figure'),
    [Input('lot-dropdown', 'value')]
)
def update_graph(selected_lot):
    # Filter data for the selected lot
    historical_data = daily_peak_usage[daily_peak_usage['LotNumber'] == selected_lot]
    forecast_data = forecast_df[forecast_df['LotNumber'] == selected_lot]
    
    # Create the figure
    fig = go.Figure()

    # Add historical data
    fig.add_trace(go.Scatter(
        x=historical_data['Date'],
        y=historical_data['PeakUsage'],
        mode='lines+markers',
        name='Historical Usage',
        line=dict(color='blue', dash='dash')
    ))

    # Add forecast data
    fig.add_trace(go.Scatter(
        x=forecast_data['Date'],
        y=forecast_data['PredictedPeakUsage'],
        mode='lines+markers',
        name='Forecasted Usage',
        line=dict(color='orange')
    ))
    
    # Update layout
    fig.update_layout(
        title=f"Peak Usage Forecast for Lot {selected_lot}",
        xaxis_title="Date",
        yaxis_title="Peak Usage",
        legend_title="Legend",
        template="plotly_white"
    )
    
    return fig

# Run the app
app.run_server(debug=True, port=8074)